In [ ]:
import pydeb
import numpy

In [ ]:
# Obtain Catalogue of Life identifier for the taxon of interest
# This function (get_ids) will return multiple matches if the provided name is not unique.
pydeb.infer.get_ids('Artemia urmiana')

In [ ]:
# Obtain the prior distribution of DEB parameters through phylogenetic inference
taxon = pydeb.infer.Taxon.from_col_id('eeabc9e772823a64ab9af8110d03f4f6')
prior = taxon.infer_parameters(offline_db='C:/Users/jornb/OneDrive/Code/debweb/data/20200514', add_del_M=True)

In [ ]:
import yaml
with open('obs/AghVSta2008b_stats.yaml') as f:
    AghVSta2008b_stats = yaml.safe_load(f)
with open('obs/AghVSta2008b.yaml') as f:
    AghVSta2008b = yaml.safe_load(f)
with open('obs/LartAgh2012.yaml') as f:
    LartAgh2012 = yaml.safe_load(f)
obs2target = {
    'total_offspring_per_female': 'N_i / (S_p + 1e-14)',
    'prereproductive_period': 'a_p - a_b',
    'lifespan': 'a_p - a_b + (a_m - state_at_time(a_p)[\'a\']) / (S_p + 1e-14)'
}


In [ ]:
timeseries_obs = LartAgh2012  #AghVSta2008b
f_egg = 0.6

# Body temperature to use for prior information and simulations
T = timeseries_obs[0]['temperature'] #taxon.typical_temperature

sampler = pydeb.calibrate.MCMCSampler()
sampler.likelihood.add_parameters(prior.names, prior.mean, prior.cov, prior.inverse_transforms)

ip_M = prior.names.index('p_M')
sampler.likelihood.add_parameters('salt_threshold', 100, 100)
sampler.likelihood.add_parameters('p_T_at_s300', prior.mean[ip_M], prior.cov[ip_M, ip_M], numpy.exp)
sampler.likelihood.add_parameters('p_T_ref', prior.mean[ip_M], prior.cov[ip_M, ip_M], numpy.exp)

class SaltDependentModel(pydeb.Model):
    def __init__(self, salinity, **params):
        super().__init__()
        for name in prior.names:
            setattr(self, name, params[name])
        p_T_ref = params['p_T_ref']
        p_T_at_s300 = params['p_T_at_s300']
        self.p_T  = p_T_ref + max(salinity - params['salt_threshold'], 0.) * (p_T_at_s300 - p_T_ref) / (300. - params['salt_threshold'])

class SalinityDependent(pydeb.calibrate.likelihood.Component):
    def __init__(self, salinity):
        super().__init__()
        self.salt = salinity

    def evaluate(self, values) -> float:
        model = SaltDependentModel(self.salt, **values)
        model.initialize()
        values['model_%i' % self.salt] = model
        if model.valid:
            return super().evaluate(values | {'model': model})

salt2comp = {}
def get_component(salinity):
    if salinity not in salt2comp:
        salt2comp[salinity] = SalinityDependent(salinity=salinity)
        if salinity < 100:
            salt2comp[salinity].add_child(pydeb.calibrate.likelihood.ImpliedProperty('L_i/del_M', 1.5, 0.1, temperature=info['temperature']))
        if salinity < 400:
            sampler.likelihood.add_child(salt2comp[salinity])
    return salt2comp[salinity]

for info in AghVSta2008b_stats:
    if info['salinity'] > 110:
        continue

    comp = get_component(info['salinity'])

    # Add observed life history traits (ultimate structural length and reproduction rate)
    for obsname, target in obs2target.items():
        comp.add_child(pydeb.calibrate.likelihood.ImpliedProperty(target, info[obsname], info['%s_sd' % obsname], temperature=info['temperature']))

def set_E_0(model, kwargs, c_T, ref_model, **params):
    kwargs['E_0'] = ref_model.E_0_at_f(f_egg)

def model_time_to_obs(model, result, c_T, **params):
    result['t'] -= model.a_b / c_T

for info in timeseries_obs:
    comp = get_component(info['salinity'])

    ts = pydeb.calibrate.likelihood.TimeSeries(info['time'][1:], temperature=info['temperature'], preprocess=set_E_0, postprocess=model_time_to_obs)
    ts.add_external_parameter('ref_model', 'model_75')
    ts.add_series('S', info['survival'][1:], sd=info['survival_sd'][1:])
    ts.add_series('L/del_M', 0.1 * numpy.array(info['length'])[1:], sd=0.1 * numpy.array(info['length_sd'])[1:])
    comp.add_child(ts)

    comp.add_child(pydeb.calibrate.likelihood.ImpliedProperty('L_b/del_M', 0.05, 0.01, temperature=info['temperature']))

bar = pydeb.calibrate.ProgressBar()
display(bar.widget)
samples = sampler.get_samples(500000, progress_reporter=bar, nburn=100000)

In [ ]:
best_lnl = -numpy.inf
for sample in samples:
    if sample['lnl'] > best_lnl:
        best_sample, best_lnl = sample, sample['lnl']
print('Highest ln likelihood witin ensemble = %s' % best_lnl)

import scipy.optimize
parnames = sampler.likelihood.get_combined_prior()[0]
def f(x):
    lnl = sampler.likelihood.evaluate(dict(zip(parnames, x)))
    return numpy.inf if lnl is None else -lnl
opt = scipy.optimize.minimize(f, [best_sample['mu'][n] for n in parnames], method='Nelder-Mead')
opt_sample = dict(zip(parnames, opt['x']))
opt_lnl = sampler.likelihood.evaluate(opt_sample)
print('Optimum ln likelihood after further optimization = %s' % opt_lnl)

for name in sampler.likelihood.get_combined_prior()[0]:
    values = [sample[name] for sample in samples]
    perc025, perc500, perc975 = numpy.percentile(values, (2.5, 50, 97.5))
    print('%s\t%s\t%s\t%.4g (%.4g - %.4g)\t%.4g' % (pydeb.long_names.get(name, name), pydeb.units.get(name, '?'), name, perc500, perc025, perc975, opt_sample[name]))



In [ ]:
bar = pydeb.calibrate.ProgressBar()
display(bar.widget)
outputs = ['L/del_M', 'S', 'R', 'N_RS']
salt2result = {}
for info in timeseries_obs:
    salt = int(info['salinity'])
    if salt not in salt2result:
        r = pydeb.simulate_ensemble([sample['model_%i' % salt] for sample in samples], outputs, T=T, t_end=90., progress_reporter=bar, t_offset='a_b', max_delta_t=0.05, E_0=sample['model_75'].E_0_at_f(f_egg))
        stats = {}
        for key, values in r.items():
            stats[key]  = values if key == 't' else numpy.percentile(values, [2.5, 25, 50, 75, 97.5], axis=0)
        salt2result[salt] = stats

In [ ]:
from matplotlib import pyplot

def add_series(ax, result, name, title=None, scale_factor=1., color='C0', label=None, opt_res=None):
    if title is None:
        title = '%s (%s)' % (pydeb.long_names.get(name, name), pydeb.units.get(name, '?'))
    p025, p250, p500, p750, p975 = [v * scale_factor for v in result[name]]
    time = numpy.array(result['t'])
    ax.fill_between(time, p025, p975, alpha=0.3, color=color)
    ax.fill_between(time, p250, p750, alpha=0.3, color=color)
    ax.plot(time, p025, '-', color=color, lw=.2)
    ax.plot(time, p975, '-', color=color, lw=.2)
    ax.plot(time, p250, '-', color=color, lw=.2)
    ax.plot(time, p750, '-', color=color, lw=.2)
    ax.plot(time, p500, '-', color=color, label=label)
    if opt_res is not None and True:
        ax.plot(opt_res[0], opt_res[1], '-', color='k')
        ax.plot(opt_res[0], opt_res[1], '--', color=color)
    ax.set_ylabel(title)
    ax.grid(True)
    ax.set_xlim(0, 70)

def create_plot(ax, name, title=None, scale_factor=1., obs_name=None):
    for i, salt in enumerate(sorted(salt2result)):
        color = 'C%i' % i

        opt_res = {}
        m = opt_sample['model_%i' % salt]
        c_T = m.get_temperature_correction(T)
        a_b = m.evaluate('a_b', c_T=c_T)
        t = 90 + a_b
        dt = .01
        r = m.simulate(int(t / dt), dt, c_T=c_T) # f_egg=f_egg)
        opt_res = (r['t'] - a_b, scale_factor * m.evaluate(name, c_T=c_T, locals=r))

        add_series(ax, salt2result[salt], name, title, scale_factor=scale_factor, color=color, label='salinity=%i' % salt, opt_res=opt_res)
        if obs_name is not None:
            for info in timeseries_obs:
                if int(info['salinity']) == salt:
                    break
            ax.errorbar(info['time'], info[obs_name], info['%s_sd' % obs_name], fmt='ow', mec=color, ecolor=color)
    ax.legend()


#for name in sim.selected_outputs:
#    data, layout = create_plot(result, name)
#    fig = go.Figure(data=data, layout=layout)
#    py.iplot(fig)
fig, (axgrowth, axsurv) = pyplot.subplots(ncols=2, figsize=(10, 5), sharex=True)
create_plot(axgrowth, 'L/del_M', 'length (mm)', scale_factor=10, obs_name='length')
create_plot(axsurv, 'S', 'survival (1)', obs_name='survival')
axgrowth.set_xlabel('time since hatching (d)');
axsurv.set_xlabel('time since hatching (d)');
fig2, (axrep, axcumr) = pyplot.subplots(nrows=2, figsize=(7, 7), sharex=True)
create_plot(axrep, 'R', 'reproduction rate (# d-1)')
create_plot(axcumr, 'N_RS', 'lifetime reproductive output (#)')
axcumr.set_xlabel('time since hatching (d)');
fig.savefig('timeseries.png', dpi=300)


In [ ]:
T_traits = 27
targets = set(obs2target.values()) | {'N_i', 'a_m', '10*L_i/del_M'}
bar = pydeb.calibrate.ProgressBar()
display(bar.widget)
salts = numpy.linspace(0, 400, 100)
target2default = {'a_p - a_b': numpy.inf}
target2values = dict([(target, numpy.zeros((len(samples), salts.size))) for target in targets])
sample2index = {}
for i, sample in enumerate(samples):
    bar(i / len(samples), '')
    key = tuple([sample['mu'][n] for n in parnames])
    if key in sample2index:
        # this parameter set has been processed before - copy previous results directly
        for values in target2values.values():
            values[i, :] = values[sample2index[key], :]
    else:
        # this parameter set needs to be processed
        sample2index[key] = i
        for isalt, salt in enumerate(salts):
            model = SaltDependentModel(salt, **sample)
            model.initialize()
            for target in targets:
                target2values[target][i, isalt] = target2default.get(target, 0.) if not model.valid else model.evaluate(target, c_T=model.get_temperature_correction(T_traits))

In [ ]:
target2optvalues = dict([(target, numpy.zeros((salts.size,))) for target in targets])
for isalt, salt in enumerate(salts):
    model = SaltDependentModel(salt, **opt_sample)
    model.initialize()
    for target in targets:
        target2optvalues[target][isalt] = target2default.get(target, 0.) if not model.valid else model.evaluate(target, c_T=model.get_temperature_correction(T))

In [ ]:
target2title = {
    'N_i': 'lifetime reproductive output (#)',
    'a_m': 'expected lifespan at birth (d)',
    '10*L_i/del_M': 'ultimate length (mm)',
    obs2target['total_offspring_per_female']: 'total offspring per mature female (#)',
    obs2target['prereproductive_period']: 'length of prereproductive period (d)',
    obs2target['lifespan']: 'expected lifespan of mature individual (d)'}
target2range = {obs2target['lifespan']: (0, 90), obs2target['prereproductive_period']: (0, 50)}
color = 'C0'

def plot_target(target, ax, label=True, panel=None):
    salt2values = target2values[target]
    p025, p250, p500, p750, p975 = numpy.percentile(salt2values, (2.5, 25, 50, 75, 97.5), axis=0)
    ax.fill_between(salts, p025, p975, alpha=0.3, color=color)
    ax.fill_between(salts, p250, p750, alpha=0.3, color=color)
    print(target, p500[0], p025[0], p975[0])
    print(target, numpy.interp(175, salts, p500), numpy.interp(175, salts, p025), numpy.interp(175, salts, p975))
    print(target, numpy.interp(350, salts, p500), numpy.interp(350, salts, p025), numpy.interp(350, salts, p975))
    ax.plot(salts, p500, color=color)
    ax.grid(True)

    ax.plot(salts, target2optvalues[target], '-k')
    ax.plot(salts, target2optvalues[target], '--', color=color)

    title = target2title.get(target, target)
    ax.set_xlim(salts[0], salts[-1])
    if label:
        ax.set_xlabel('salinity')
    ax.set_ylabel(title)
    ylim = target2range.get(target, (None, None))
    for obsname, obstarget in obs2target.items():
        if obstarget == target:
            for info in AghVSta2008b_stats:
                if info['salinity'] < 110:
                    ax.errorbar(info['salinity'], info[obsname], info['%s_sd' % obsname], fmt='ow', mec='k', ecolor='k')
    ax.set_ylim(*ylim)
    if panel is not None:
        ax.text(.05, .05, panel, transform=ax.transAxes, fontweight='bold', fontsize=14)

# for target in target2values:
#     fig, ax = pyplot.subplots()
#     plot_target(target, ax)
#     fig.savefig('%s.png' % (title.split(' (', 1)[0].replace(' ', '_'),), dpi=300)
        
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = pyplot.subplots(nrows=3, ncols=2, figsize=(8, 10), sharex=True)
plot_target('10*L_i/del_M', ax1, label=False, panel='a')
plot_target('a_m', ax2, label=False, panel='b')
plot_target('N_i', ax3, label=False, panel='c')
plot_target(obs2target['lifespan'], ax4, label=False, panel='d')
plot_target(obs2target['prereproductive_period'], ax5, panel='e')
plot_target(obs2target['total_offspring_per_female'], ax6, panel='f')
ax3.set_ylim(0, None)
ax6.set_ylim(0, None)
fig.tight_layout()
fig.savefig('deb_traits.png', dpi=300)

In [ ]:
opt_sample['model_75'].writeFABMConfiguration('fabm.yaml')

In [ ]:
for key in opt_sample:
    if key.startswith('model_'):
        print(key, opt_sample[key].E_0 * opt_sample[key].kap_R)

In [ ]:
salt = 75
opt_res = {}
m = opt_sample['model_%i' % salt]
c_T = m.get_temperature_correction(T)
a_b = m.evaluate('a_b', c_T=c_T)
t = 1
dt = 0.1
y_ini = None
t0 = 0
rs = {}
for iday in range(23):
    salinity = min(35 + iday*30, 200)
    salinity = 200
    p_T_ref = opt_sample['p_T_ref']
    p_T_at_s300 = opt_sample['p_T_at_s300']
    m.p_T  = p_T_ref + max(salinity - opt_sample['salt_threshold'], 0.) * (p_T_at_s300 - p_T_ref) / (300. - opt_sample['salt_threshold'])
    m.initialize()
    print(m.p_T)
    r = m.simulate(int(t / dt), dt, c_T=c_T, y_ini=y_ini)
    r['t'][:] += t0
    y_ini = dict([(n, r[n][-1]) for n in r])
    t0 = r['t'][-1]
    for n in r:
        rs[n] = numpy.concatenate((rs.get(n, []), r[n]))
rs['S'][-1]



In [ ]:
from matplotlib import pyplot
fig, ax = pyplot.subplots()
ax.plot(rs['t'], rs['S'])
ax.set_ylim(0, 1)

In [ ]:
rs['S']

In [ ]:
y_ini = None
t=900
r = m.simulate(int(t / dt), dt, c_T=c_T, y_ini=y_ini)
from matplotlib import pyplot
fig, ax = pyplot.subplots()
ax.plot(r['t'], r['H'] / r['L']**3)